In [1]:
# Connect to webhose, get your API key and do the following, to retrieve the dataset.
import json
# import word2vec
import gensim, operator,sys
from scipy import spatial
import numpy as np
from gensim.models import KeyedVectors
import watson_developer_cloud
import pprint
import webhoseio

c:\python36\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
webhoseio.config(token='8e242357-4a54-4645-a4ef-5ce38b0b9c7d ')
query_params = {
    'q': 'organization:Apple',
    'ts': '1511056980199',
    'sort': 'crawled'
}

results = webhoseio.query('filterWebContent', query_params)

In [3]:
with open('anly_ner.json', 'w', encoding='utf8') as outfile:
    outfile.write(json.dumps(results, indent=2))

In [4]:
myfile =  open('anly_ner.json','r')
mydata = json.loads(myfile.read())

titles = set()
for post in mydata['posts']:
    if post['title']:
        titles.add(post['title'])

titles = list(titles)
mydata_titles = ' '.join(titles)

In [5]:
import json
from ibm_watson import NaturalLanguageUnderstandingV1
from watson_developer_cloud.natural_language_understanding_v1 import Features, EntitiesOptions
natural_language_understanding = NaturalLanguageUnderstandingV1(iam_apikey='tkYa9Q8orVp7Mn3zrkA7DOh0Ic-VfgZL2a7kHWqjrn-Y',
  version="2017-11-20")
response = natural_language_understanding.analyze(
   text = mydata_titles,
  features=Features(entities=EntitiesOptions())
)
# #print(json.dumps(response, indent=2))

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import nltk, re
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
import json

stopwords = set(nltk.corpus.stopwords.words('english'))
tokenizer = RegexpTokenizer(r'\w+')

In [7]:
def tokenize(text):
    
#first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens  = [word for sent in nltk.sent_tokenize(text) \
              for word in tokenizer.tokenize(sent) if word not in stopwords]
    lmtzr   = WordNetLemmatizer()

    filtered_tokens = []
   # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
         if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token.lower())
    
    stems = [lmtzr.lemmatize(t) for t in filtered_tokens]
    return stems


def clstr_lda(num_topics, stories):
    """
    :rtype: object
    """
    n_top_words = 15

    tf_vectorizer = CountVectorizer(tokenizer=tokenize)
    tf = tf_vectorizer.fit_transform(stories)

    lda = LatentDirichletAllocation(n_components=num_topics, max_iter=200,
                                    learning_method='online', learning_offset=9.,
                                    random_state=1)
    lda.fit(tf)
    tf_feature_names = tf_vectorizer.get_feature_names()

    #print("\nLDA Topics:")
    # print top topic words
    topics = dict()
    for topic_idx, topic in enumerate(lda.components_):
        topics[topic_idx] = [tf_feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
        print("Topic #%d:" % topic_idx)
        print(" | ".join([tf_feature_names[i]
                         for i in topic.argsort()[:-n_top_words - 1:-1]]))
    #print()
    return topics

In [8]:
myfile =  open('anly_ner.json','r')
mydata = json.loads(myfile.read())

texts = list()
for post in mydata['posts']:
    texts.append(post['title'])

topics = clstr_lda(8, texts)

Topic #0:
industrials | fall | roundup | mexico | tariff | measure | may | applecare | headphone | jack | galaxy | note | samsung | online | four
Topic #1:
iphone | apple | ride | subway | air | download | 200mb | today | best | deal | company | rocketman | free | city | future
Topic #2:
apple | year | itunes | new | end | apps | mac | next | bring | computer | era | feature | later | plan | tech
Topic #3:
apple | itunes | apps | limit | tracking | third | party | week | report | android | main | reportedly | kid | donepezil | card
Topic #4:
wwdc | apple | expect | iphone | china | dominate | looking | xiaomi | monday | keynote | redmi | say | one | conference | wither
Topic #5:
apple | new | wwdc | macos | news | york | google | saatchi | tvt | effie | double | entendre | show | year | expect
Topic #6:
app | apple | download | mb | limit | store | io | superfast | prepare | 5g | ridiculous | bump | ipod | touch | still
Topic #7:
apple | mac | ipad | di | closer | ever | poised | bring

In [9]:
output = open('results.txt','w')
for topic_idx, topic in topics.items():
    output.write("Topic #%d: \n" % topic_idx)
    output.write(" | ".join(topic) + '\n')
        
output.close()

In [10]:
topic_taxonomy = {
    "offers":
    {
        "free": "shipping trial product",
        "deal": "discount countdown limited ",
        "unlimited": "stock warranty buy"
    },
    "companies":
    {
        "amazon": "shopping order department",
        "facebook": "likes social advertisement",
        "google": "maps search doodle"
    },
    "returns":
    {
        "shipped": "free checkout prime",
        "cashback": "credit cash days",
        "balance": "card money transfer"
    }
}

In [11]:
import gensim,operator
from scipy import spatial
import numpy as np
from gensim.models import KeyedVectors

model_path =  "watson_nlp_model/"

def load_word2vec_model(modelName, modelFile, flagBin):
    print('Loading ' + modelName + ' model...')
    model = KeyedVectors.load_word2vec_format(model_path + modelFile, binary=flagBin)
    print('Finished loading ' + modelName + ' model...')
    return model
model_word2vec = load_word2vec_model('word2Vec', 'GoogleNews-vectors-negative300.bin', True)

Loading word2Vec model...
Finished loading word2Vec model...


In [12]:
def vocab_check(vectors,words):
    output = list()
    for word in words:
        if word in vectors.vocab:
            output.append(word.strip())
            
    return output

#function calculating similarity between two strings :
def calc_similarity(input1, input2, vectors):
    s1words = set(vocab_check(vectors, input1.split()))
    s2words = set(vocab_check(vectors, input2.split()))
    if len(s1words) > 0 and len(s2words) > 0:
        return vectors.n_similarity(s1words, s2words)
    else:
        return 0

for i in range(len(titles)):
    target = titles[i]
    scores = []
    subtitles = titles.copy()
    subtitles.pop(i)
    for title in subtitles:
        scores.append((title,calc_similarity(target, title, model_word2vec)))
    
    sorted_scores = sorted(scores, key=lambda x: x[1], reverse=True)
    
    print('Target Title: ', target)
    print('----------------------------------------------------------')
    for j in range(10):
        print(sorted_scores[j][0])
        
    print('\n\n')

Target Title:  J.K. Rowling is releasing four new Harry Potter books online
----------------------------------------------------------
All the new features Apple is expected to bring to your Mac computer later this year
Apple will reportedly limit third-party tracking in children's apps
All the new features Apple is expected to bring to your Mac computer later this year,
Apple is finally going to put iTunes out of its misery
Apple might be killing iTunes this year
Apple is eliminating ‘Back to My Mac’ in all macOS versions in July
Apple will reportedly limit third-party tracking in children’s apps
The surprising main reason 492 out of 500 people at one company chose iPhone over Android
Apple to limit third-party tracking in kids apps, report says
This 14-year-old boy is missing from Trafford - police are becoming 'increasingly concerned'



Target Title:  Bumpy car rides suck and a newly published patent reveals Ap
----------------------------------------------------------
Finally, I c

c:\python36\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


Target Title:  " Earlier this week the iPod touch received a modest upgrade, arriving with little fanfare via an..."
----------------------------------------------------------
All the new features Apple is expected to bring to your Mac computer later this year
All the new features Apple is expected to bring to your Mac computer later this year,
" Whether you’re heading to WWDC and other coinciding conferences in San Jose this year for the first..."
Save more than 60% on this charging dock for your iPhone, Apple Watch, and AirPods
" Have you already purchased a Mac but now decided that you’d like to protect it with AppleCare? Keep..."
This week in Apple: The top news from Android’s main competitor
Eli Apple on not having 2020 option picked up: Have to go out and prove myself
Google quietly ruined Chrome, and we almost missed it
Apple is finally going to put iTunes out of its misery
Apple might be killing iTunes this year



Target Title:  Foxconn Moves the Apple Production Line Out of C

Target Title:  Buying donepezil online legal traffic, cheap donepezil fast
----------------------------------------------------------
iOS apps reportedly sharing massive amounts of data with tracking companies when you sleep
6 paid iPhone apps on sale for free on May 31st
" Have you already purchased a Mac but now decided that you’d like to protect it with AppleCare? Keep..."
You can now download files up to 200MB over the air on your iPhone
How to add AppleCare to your Mac after purchase
Here’s how to stand out in a crowded streaming service market
To prepare for superfast 5G, Apple bumps iOS App Store download limit to a ridiculous 200 MB
Best laptop and tablet deals this weekend: Microsoft Surface, Lenovo Tab, iPad, and others
Apple quietly boosts App Store cellular download limit to 200MB
Apple increases cellular download limit for iPhones and iPads



Target Title:  "Neben dem Ausbau des spiegellosen Vollformats macht Panasonic deutlich, dass das..."
------------------------------

Target Title:  “It’s a Tide Ad” by Procter & Gamble & Saatchi & Saatchi New York wins Grand Effie at the 2019 Effie Awards U.S.
----------------------------------------------------------
New York Genuine Estate The Empire State 16221
New iPod Touch Delivers Even Greater Performance
Tech Drops as China Prepares Response to U.S. Huawei Moves -- Tech Roundup
Best Free Games for iPod Touch
Qualcomm Price Target Cut During Worst Month in Years
Apple Plans End of iTunes, to Reveal Glimpses of Its Next Era of Apps and Devices
Fandango Apple Pay Promotion: Get $5 Off Two Rocketman Tickets w/ Promo Code ROCKETMAN
Google Requires Loot Box-Laden Kids’ Games Disclose the Odds of Winning Digital Goods
Issue 677: Apple’s Newest Innovation is… the iPod; After 15 Years, the Pirate Bay Still Can’t Be Killed
Huawei ban : China to hit back at U.S with its own entity list



Target Title:  Fandango Apple Pay Promotion: Get $5 Off Two Rocketman Tickets w/ Promo Code ROCKETMAN
------------------------------

Target Title:  Apple is shutting down iTunes
----------------------------------------------------------
Apple might be killing iTunes this year
Apple is finally going to put iTunes out of its misery
Apple Music may replace iTunes at WWDC 2019
Apple is finally ditching iTunes.
Everything Coming to iTunes & the Apple TV App in June 2019
Apple increases cellular download limit for iPhones and iPads
Apple is eliminating ‘Back to My Mac’ in all macOS versions in July
Apple quietly boosts App Store cellular download limit to 200MB
Apple will reportedly limit third-party tracking in children’s apps
Apple will reportedly limit third-party tracking in children's apps



Target Title:  Apple will reportedly limit third-party tracking in children’s apps
----------------------------------------------------------
Apple will reportedly limit third-party tracking in children's apps
Apple to limit third-party tracking in kids apps, report says
iOS apps reportedly sharing massive amounts of data with t

Target Title:  Motorola says no Z4 Force this year as flagship ambitions wither
----------------------------------------------------------
All the new features Apple is expected to bring to your Mac computer later this year
The surprising main reason 492 out of 500 people at one company chose iPhone over Android
All the new features Apple is expected to bring to your Mac computer later this year,
Apple is finally going to put iTunes out of its misery
Apple is eliminating ‘Back to My Mac’ in all macOS versions in July
Apple might be killing iTunes this year
Huawei ban : China to hit back at U.S with its own entity list
" Have you already purchased a Mac but now decided that you’d like to protect it with AppleCare? Keep..."
Google quietly ruined Chrome, and we almost missed it
Save 53% on this 3-in-1 charging stand for the ultimate Apple fan



Target Title:  How to add AppleCare to your Mac after purchase
----------------------------------------------------------
All the new features Ap

The surprising main reason 492 out of 500 people at one company chose iPhone over Android
All the new features Apple is expected to bring to your Mac computer later this year
Apple increases cellular download limit for iPhones and iPads
We used our iPhone to ride the New York City subway today – it's the future
To prepare for superfast 5G, Apple bumps iOS App Store download limit to a ridiculous 200 MB
You can now download files up to 200MB over the air on your iPhone
Everything Coming to iTunes & the Apple TV App in June 2019
Apple will reportedly limit third-party tracking in children’s apps
Apple will reportedly limit third-party tracking in children's apps



Target Title:  Save 53% on this 3-in-1 charging stand for the ultimate Apple fan
----------------------------------------------------------
Save more than 60% on this charging dock for your iPhone, Apple Watch, and AirPods
All the new features Apple is expected to bring to your Mac computer later this year,
All the new feature

In [13]:
def vec_similarity(input1, input2, vectors):
    term_vectors = [np.zeros(300), np.zeros(300)]
    terms = [input1, input2]
        
    for index, term in enumerate(terms):
        for i, t in enumerate(term.split(' ')):
            try:
                term_vectors[index] += vectors[t]
            except:
                term_vectors[index] += 0
        
    result = (1 - spatial.distance.cosine(term_vectors[0], term_vectors[1]))
    if result is 'nan':
        result = 0
        
    return result

In [14]:
# function checks whether the input words are present in the vocabulary for the model
def vocab_check(vectors, words):
    
    output = list()
    for word in words:
        if word in vectors.vocab:
            output.append(word.strip())
            
    return output

In [15]:
def calc_similarity(input1, input2, vectors):
    s1words = set(vocab_check(vectors, input1.split()))
    s2words = set(vocab_check(vectors, input2.split()))
    
    output = vectors.n_similarity(s1words, s2words)
    return output

In [16]:
# function takes an input string, runs similarity for each item in topic_taxonomy, sorts and returns top 3 results
def classify_topics(input, vectors):
    feed_score = dict()
    for key, value in topic_taxonomy.items():
        max_value_score = dict()
        for label, keywords in value.items():
            max_value_score[label] = 0
            topic = (key + ' ' + keywords).strip()
            max_value_score[label] += float(calc_similarity(input, topic, vectors))
            
        sorted_max_score = sorted(max_value_score.items(), key=operator.itemgetter(1), reverse=True)[0]
        feed_score[sorted_max_score[0]] = sorted_max_score[1]
    return sorted(feed_score.items(), key=operator.itemgetter(1), reverse=True)[:3]

In [17]:
output = open('classified topics.txt','w')
for post in mydata['posts']:
        output.write(str(str(post["title"]).encode(encoding='ascii',errors='ignore')))
        output.write('\n')
        try:
            output.write(str(classify_topics(post["title"], model_word2vec))+'\n\n')
        except ZeroDivisionError:
            output.write('0')
        
output.close()

c:\python36\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [18]:
print('----Done-----------')

----Done-----------
